In [1]:
from __future__ import division
import tensorflow as tf
from Ops import *

/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'Ops'

####  discriminator 

- Q1. why discriminator's output is 32x32x1 
> I thought discriminator has a role of discriminating fake or real with 0, 1 each.
> is it technic ? 

>> **This Answer was writen on the paper**
>> - by implementing, reserchers dose Johnson 's Residual architecture which is known as very productive on style transfering problem for generator, and use **PatchGAN**'s discriminator 


###### PatchGAN research 

 > blah blah
 

In [ ]:

def discriminator(image, output_dim=8, reuse=False, name="discriminator"):
    with tf.variable_scope(name):
        if reuse 
            tf.get_variable_scope().reuse_variables()
        else :
            assert tf.get_variable_scope().reuse is False
        # h0 is (150 x 150 x self.df_dim)
        h0 = leaky_relu(conv2d(image, output_dim, name='dis_h0_conv'))
        # h1 is (75 x 75 x self.df_dim*2)
        h1 = lrelu(instance_norm(conv2d(h0, output_dim*2, name='d_h1_conv'), 'd_in1'))
        # h2 is (36 x 36 x self.df_dim*4)
        h2 = lrelu(instance_norm(conv2d(h1, output_dim*4, name='d_h2_conv'), 'd_in2'))
        # h3 is (36 x 36 x self.df_dim*8)
        h3 = lrelu(instance_norm(conv2d(h2, output_dim*8, stride=1, name='d_h3_conv'), 'd_in3'))
        # h4 is (36 x 36 x 1)
        h4 = conv2d(h3, 1, stride=1, name='d_h3_pred')

        return h4

In [ ]:
def generator_dense(image, reuse=False, name="generator"):
    with tf.variable_scope(name):
        # image = 300x300x1
        
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False
            
        def dense_block(x, growth_rate=8, num_layers_in_block=6, name="dense"):
            
            for i in range(num_layers_in_block):    
                y1 = instance_norm(conv2d(x, growth_rate*2, 
                                          kernel_size=1, stride=1, 
                                          padding='SAME', 
                                          name=name+'_bot{}'.format(i)), name+'_in1{}'.format(i))
                y2 = instance_norm(conv2d(y1, growth_rate, 
                                          kernel_size=3, stride=1, 
                                          padding='SAME', 
                                          name=name+'_tle{}'.format(i)), name+'_in2{}'.format(i))
                x = tf.concat(y2, x)
                
                return x 
                
                
        dense_layer_1 = dense_block(image, name="dense_1")
        
        # Maintaing Feature map size , Reduce # of channels  
        transition_layer = instance_norm(conv2d(dense_layer_1, 1, 
                                          kernel_size=3, stride=1, 
                                          padding='SAME', 
                                          name=name+'_transition1'), name+'_in1')
        
        dense_layer_2 = dense_block(transition_layer, name="dense_2")
        
        
        output_layer = instance_norm(conv2d(dense_layer_2, 1, 
                                          kernel_size=3, stride=1, 
                                          padding='SAME', 
                                          name=name+'_out'), name+'_in_out')
        
        
        return output_layer


In [ ]:
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))


def mse_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)


def sce_criterion(logits, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
        